# Análise da relação de requisitos técnicos entre si

In [1]:
import networkx as nx
import numpy as np
import pandas as pd

In [2]:
# Importando o grafo completo
G = nx.read_gexf("job_skill_graph.gexf")

In [3]:
# Analisando os tipos de nós
node_types = nx.get_node_attributes(G, "type")
print("Tipos de nós no grafo:", set(node_types.values()))

Tipos de nós no grafo: {'job', 'skill'}


In [4]:
# Calculando o total de nós do tipo 'job'
total_jobs = len({n for n, t in node_types.items() if t == "job"})
print("Total de nós do tipo 'job':", total_jobs)

Total de nós do tipo 'job': 55544


## Realizando a projeção no grafo completo para relacionar requisitos entre si

In [5]:
skills = {n for n, t in node_types.items() if t == "skill"}

use_available_data = True
if not use_available_data:
    # Pesos: número de vizinhos em comum (número de vagas que requerem ambos os requisitos)
    G_skills_simple_weight = nx.bipartite.weighted_projected_graph(
        G, skills, ratio=False
    )
else:
    G_skills_simple_weight = nx.read_gexf(
        "subgraphs/projection_skills_graph_simple_weight.gexf"
    )

## Executando um algoritmo de detecção de comunidades

In [6]:
def normalize_feature(feature):
    """
    Normalizes a dictionary of node features to a range between 0 and 1 (min-max normalization).
    Args:
        feature (dict): A dictionary with node IDs as keys and feature values as values.
    Returns:
        dict: A dictionary with node IDs as keys and normalized feature values as values.
    """
    values_array = np.array(list(feature.values()))
    min_val = values_array.min()
    max_val = values_array.max()

    for node, value in feature.items():
        feature[node] = (value - min_val) / (max_val - min_val)

    return feature

In [7]:
def evaluate_top_degrees(G_skill_community, G_full_graph):
    """
    Evaluates and prints the top 5 skills by degree within a given skill community graph.
    Also adds the normalized degree as a node attribute to the full graph.
    Args:
        G_skill_community (networkx.Graph): The subgraph representing the skill community.
        G_full_graph (networkx.Graph): The full job-skill graph.
    Returns:
        None
    """
    degrees = dict(G_skill_community.degree())

    df = pd.DataFrame(
        {
            "skill": list(degrees.keys()),
            "degree": list(degrees.values()),
        }
    )

    # Adicionando grau como atributo aos nós do grafo
    nx.set_node_attributes(G_full_graph, normalize_feature(degrees), "degree")

    print("  GRAU (APENAS DENTRO DA COMUNIDADE):", end=" ")
    df = df.sort_values("degree", ascending=False).reset_index(drop=True)
    top_5_degrees = (
        df[["skill", "degree"]]
        .head(5)
        .apply(
            lambda x: f"{x['skill']}:{x['degree']} ({x['degree'] / len(df) * 100:.2f}%)",
            axis=1,
        )
        .to_frame()
        .T
    )
    print(top_5_degrees.to_string(index=False, header=False))

In [8]:
def number_of_related_jobs(G_skill_community, G):
    """
    Counts the number of unique jobs related to the skills in the given skill community graph.
    Args:
        G_skill_community (networkx.Graph): The subgraph representing the skill community.
        G (networkx.Graph): The full job-skill graph.
    Returns:
        int: The number of unique related jobs.
    """
    related_jobs = set()
    for skill in G_skill_community.nodes():
        neighbors = G.neighbors(skill)
        related_jobs.update([n for n in neighbors if node_types[n] == "job"])
    return len(related_jobs)

In [9]:
# Executando um algoritmo de detecção de comunidades
if not use_available_data:
    simple_weight_communities = nx.community.louvain_communities(
        G_skills_simple_weight, weight="weight", seed=859
    )

    # Inserindo um atributo 'community' em cada nó do grafo projetado
    for i, community in enumerate(simple_weight_communities):
        for node in community:
            G_skills_simple_weight.nodes[node]["community"] = i + 1
else:
    node_communities = nx.get_node_attributes(G_skills_simple_weight, "community")

    # Separando as comunidades já encontradas
    communities = {}
    for node, community_id in node_communities.items():
        if community_id not in communities:
            communities[community_id] = set()
        communities[community_id].add(node)
    # Ordenando pelo número da comunidade
    sorted_communities = dict(sorted(communities.items()))
    simple_weight_communities = list(sorted_communities.values())

In [10]:
# Verificando a modularidade
modularity = nx.community.modularity(
    G_skills_simple_weight, simple_weight_communities, weight="weight"
)
print(f"Modularidade da partição: {modularity:.4f}")

# Analisando métricas em cada comunidade
print(f"Número de comunidades encontradas: {len(simple_weight_communities)}")
for i, community in enumerate(simple_weight_communities):
    subgraph = G_skills_simple_weight.subgraph(community)
    total_jobs_related = number_of_related_jobs(subgraph, G)
    print(
        f"Comunidade {i+1}: {len(subgraph)} nós e {total_jobs_related} vagas relacionadas ({100 * total_jobs_related / total_jobs:.2f}%)"
    )
    evaluate_top_degrees(subgraph, G_skills_simple_weight)

Modularidade da partição: 0.1704
Número de comunidades encontradas: 5
Comunidade 1: 131 nós e 24601 vagas relacionadas (44.29%)
  GRAU (APENAS DENTRO DA COMUNIDADE): java:124 (94.66%) git:122 (93.13%) react:120 (91.60%) javascript:120 (91.60%) html:115 (87.79%)
Comunidade 2: 172 nós e 27235 vagas relacionadas (49.03%)
  GRAU (APENAS DENTRO DA COMUNIDADE): ci/cd:157 (91.28%) linux:157 (91.28%) kubernetes:147 (85.47%) docker:144 (83.72%) go:140 (81.40%)
Comunidade 3: 137 nós e 34590 vagas relacionadas (62.27%)
  GRAU (APENAS DENTRO DA COMUNIDADE): sql:133 (97.08%) python:132 (96.35%) aws:129 (94.16%) azure:128 (93.43%) gcp:118 (86.13%)
Comunidade 4: 32 nós e 8403 vagas relacionadas (15.13%)
  GRAU (APENAS DENTRO DA COMUNIDADE): photoshop:25 (78.12%) figma:23 (71.88%) illustrator:22 (68.75%) indesign:21 (65.62%) sketch:20 (62.50%)
Comunidade 5: 60 nós e 20317 vagas relacionadas (36.58%)
  GRAU (APENAS DENTRO DA COMUNIDADE): scrum:58 (96.67%) kanban:57 (95.00%) c#:55 (91.67%) agile:55 (91.

## Analisando as conexões mais importantes entre comunidades diferentes

In [11]:
def check_most_important_edges(G_skill_graph):
    """
    Identifies and prints the most important edges connecting different communities in the skill graph.
    Uses edge betweenness centrality to determine importance.
    Args:
        G_skill_graph (networkx.Graph): The skill graph with community information as node attributes.
    Returns:
        None
    """
    # Arestas com um peso alto significam que os nós estão mais fortemente conectados
    # A distância deles deve ser considerada menor, portanto, invertemos o peso para calcular a "distância"
    inverted_weights = {
        (u, v): 1 / data["weight"] for u, v, data in G_skill_graph.edges(data=True)
    }
    nx.set_edge_attributes(G_skill_graph, inverted_weights, "inverted_weight")

    edge_betweenness = nx.edge_betweenness_centrality(
        G_skill_graph, weight="inverted_weight"
    )

    # Considerando apenas arestas que conectam comunidades diferentes
    inter_community_edges = {
        (u, v): bw
        for (u, v), bw in edge_betweenness.items()
        if G_skill_graph.nodes[u]["community"] != G_skill_graph.nodes[v]["community"]
    }

    # Ordenando as arestas pelo betweenness centrality
    sorted_edges = sorted(
        inter_community_edges.items(), key=lambda item: item[1], reverse=True
    )

    print("ARESTAS MAIS IMPORTANTES PARA CONECTAR COMUNIDADES:")
    for (u, v), bw in sorted_edges[:3]:
        print(
            f"Aresta ({u}, {v}) com betweenness centrality {bw:.5f} liga as comunidades {G_skill_graph.nodes[u]['community']} ({u}) e {G_skill_graph.nodes[v]['community']} ({v})"
        )

    # Analisando, por comunidade, a aresta mais importante que conecta a comunidade a outras
    community_edges = {}
    for (u, v), bw in inter_community_edges.items():
        comm_u = G_skill_graph.nodes[u]["community"]
        comm_v = G_skill_graph.nodes[v]["community"]
        if comm_u not in community_edges or bw > community_edges[comm_u][2]:
            community_edges[comm_u] = (u, v, bw)
        if comm_v not in community_edges or bw > community_edges[comm_v][2]:
            community_edges[comm_v] = (v, u, bw)

    print("\nARESTAS MAIS IMPORTANTES POR COMUNIDADE:")
    community_edges = dict(sorted(community_edges.items()))
    for comm, (u, v, bw) in community_edges.items():
        print(
            f"Comunidade {comm}: Aresta ({u} - {G_skill_graph.nodes[u]['community']}, {v} - {G_skill_graph.nodes[v]['community']}) com betweenness centrality {bw:.5f}"
        )

In [12]:
# Analisando as habilidades técnicas mais relacionadas a áreas (comunidades) diferentes
check_most_important_edges(G_skills_simple_weight)

ARESTAS MAIS IMPORTANTES PARA CONECTAR COMUNIDADES:
Aresta (java, python) com betweenness centrality 0.09874 liga as comunidades 1 (java) e 3 (python)
Aresta (css, figma) com betweenness centrality 0.06923 liga as comunidades 1 (css) e 4 (figma)
Aresta (linux, python) com betweenness centrality 0.05990 liga as comunidades 2 (linux) e 3 (python)

ARESTAS MAIS IMPORTANTES POR COMUNIDADE:
Comunidade 1: Aresta (java - 1, python - 3) com betweenness centrality 0.09874
Comunidade 2: Aresta (linux - 2, python - 3) com betweenness centrality 0.05990
Comunidade 3: Aresta (python - 3, java - 1) com betweenness centrality 0.09874
Comunidade 4: Aresta (figma - 4, css - 1) com betweenness centrality 0.06923
Comunidade 5: Aresta (scrum - 5, sql - 3) com betweenness centrality 0.02176


## Exportando o grafo para visualização

In [13]:
# Salvando o grafo projetado com as comunidades e métricas para visualização
nx.write_gexf(
    G_skills_simple_weight, "subgraphs/projection_skills_graph_simple_weight.gexf"
)